# PROJECT OVERVIEW 

1.  IMPORTING REQUIRED LIBRARIES
2.  IMPORTING THE ANZ SYSTHESIZED TRANSACTIONS DATASET
3.  DATA PROFILING FOR SIMPLE VISUALISATION AND QUICK INSIGHTS ABOUT THE DATASET
4.  TREATMENT OF MISSING DATA
5.  VISUALISATION WITH PCA AND PAIRPLOT
6.  TESTING FOR CORRELATION
7.  CHECKING FOR DEPENDENTS FEATURES
8.  TESTING AND HANDLING OUTLIERS IN THE DATASET
9.  ANALYSING CUSTOMERS' CLUSTERS TO KNOW HOW MANY
10. CLUSTERING AND INTERPRETATION
11. CONCLUSION

# 1. IMPORTING REQUIRED LIBRARIES

In [1]:
#Import Libraries
import numpy as np # Linear Algebra
import pandas as pd #Loading Dataframe and Manipulations
import matplotlib as mpl # Visualisations
import matplotlib.pyplot as plt # Visualisation
import seaborn as sns #Visualisation
import pandas_profiling as pp # Data Overview
from sklearn.decomposition import PCA # Visualisation
from sklearn.cluster import KMeans # Segmentation
from sklearn.preprocessing import StandardScaler # Useful for Standardization
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import pairwise_distances
from IPython.display import display, HTML
import datetime, nltk, warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# IMPORTING THE ANZ SYNTHESIZED TRANSACTIONS DATASET

In [2]:
#Loading our dataset and assigning it a name "Data" for ease of reference
data = pd.read_excel('ANZ synthesised transaction dataset.xlsx')

In [ ]:
pp.ProfileReport(data)

In [8]:
#Viewing the Data head
data.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


# DATA PROFILING FOR SIMPLE VISUALISATION AND QUICK INSIGHTS ABOUT THE DATASET

In [9]:
# We will take a look into how the data are distributed and where we have missing values, correlation and other important information to guide our understanding of the data
Data_Profile = pp.ProfileReport(data, check_correlation = True)
Data_Profile

Number of variables,23
Number of observations,12043
Total Missing (%),15.9%
Total size in memory,2.1 MiB
Average record size in memory,184.0 B
Numeric,4
Categorical,14
Boolean,1
Date,1
Text (Unique),1
Rejected,2


Our data profile has revealed a lot about our data which will guide our next set of actions. Just to briefly outline some of the important information revealed:
1. bpay_biller_code, merchant_code have 11158 and 11160 missing values which represent about 93% respectively of the total values of the features. Definitely, it is advisable we dropped it.
2. Country and Currency both have constant values. They are not needed in our analysis since they are assumable
3. Other features with Missing values would be critically explored to understand better why they all have equal missing values. That also would help us to know the connection these features have with each others.
4. Majority of the features have high cardinality. This would be treated in our Data Cleaning Part
5. The Mean Age is 30.6, Median Age is 28. Whereas the Mininimum and Maximum Ages are 18 and 78 respectively. We would need to explore Age further for understanding.
6. We have 6285 Male and 5758 Females Customers
7. We have more Debits (11160) than Credits (883) and More Authorized transactions (7717) than Posted transactions (4326)

# TREATMENT OF MISSING DATA

In [10]:
pre_data = data.drop(['currency', 'country', 'bpay_biller_code', 'merchant_code' ], axis = 1, inplace = True)
pre_data

AttributeError: 'NoneType' object has no attribute 'head'

In [11]:
data_profile2 = pp.ProfileReport(pre_data)
data_profile2

AttributeError: 'NoneType' object has no attribute 'head'

In [12]:
pre_data["account_id"] = pre_data['account'].apply(lambda id: id.split('-')[1])

TypeError: 'NoneType' object is not subscriptable

In [13]:
pre_data.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [11]:
pre_data = pre_data.drop(['account'], axis=1)

In [12]:
pre_data.head()

,status,card_present_flag,long_lat,txn_description,merchant_id,first_name,balance,date,gender,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,customer_id,merchant_long_lat,movement,account_id
0,authorized,1.0,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,Diana,35.39,2018-08-01,F,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,CUS-2487424745,153.38 -27.99,debit,1598451071
1,authorized,0.0,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,Diana,21.20,2018-08-01,F,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,CUS-2487424745,151.21 -33.87,debit,1598451071
2,authorized,1.0,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,Michael,5.71,2018-08-01,M,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,CUS-2142601169,151.21 -33.87,debit,1222300524
3,authorized,1.0,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,Rhonda,2117.22,2018-08-01,F,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,CUS-1614226872,153.05 -26.68,debit,1037050564
4,authorized,1.0,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,Diana,17.95,2018-08-01,F,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,CUS-2487424745,153.44 -28.06,debit,1598451071


In [52]:
data.head(20)

,status,card_present_flag,account,long_lat,txn_description,merchant_id,first_name,balance,date,gender,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,customer_id,merchant_long_lat,movement,account_id
0,authorized,1.0,ACC-1598451071,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,Diana,35.39,2018-08-01,F,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,CUS-2487424745,153.38 -27.99,debit,1598451071
1,authorized,0.0,ACC-1598451071,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,Diana,21.20,2018-08-01,F,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,CUS-2487424745,151.21 -33.87,debit,1598451071
2,authorized,1.0,ACC-1222300524,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,Michael,5.71,2018-08-01,M,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,CUS-2142601169,151.21 -33.87,debit,1222300524
3,authorized,1.0,ACC-1037050564,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,Rhonda,2117.22,2018-08-01,F,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,CUS-1614226872,153.05 -26.68,debit,1037050564
4,authorized,1.0,ACC-1598451071,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,Diana,17.95,2018-08-01,F,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,CUS-2487424745,153.44 -28.06,debit,1598451071
5,posted,NaN,ACC-1608363396,151.22 -33.87,PAYMENT,NaN,Robert,1705.43,2018-08-01,M,20,NaN,NaN,2018-08-01T02:00:00.000+0000,163.00,1005b48a6eda4ffd85e9b649dc9467d3,CUS-2688605418,NaN,debit,1608363396
6,authorized,1.0,ACC-2776252858,144.95 -37.76,SALES-POS,3aa18033-a0a9-4190-a117-b7caaf4d07e3,Kristin,1248.36,2018-08-01,F,43,Kalkallo,VIC,2018-08-01T02:23:04.000+0000,61.06,b79ca208099c4c28aa5dae966096d842,CUS-4123612273,144.95 -37.53,debit,2776252858
7,authorized,1.0,ACC-2776252858,144.95 -37.76,POS,ee58145d-26e8-4b01-9cd9-62378014f21d,Kristin,1232.75,2018-08-01,F,43,Melbourne,VIC,2018-08-01T04:11:25.000+0000,15.61,e1c4a50d6a0549cbb3710a62a2fa00d5,CUS-4123612273,144.96 -37.81,debit,2776252858
8,authorized,1.0,ACC-182446574,116.06 -32.00,POS,cfbf535e-caa8-499f-9d41-bbdc2b836d85,Tonya,213.16,2018-08-01,F,27,Yokine,WA,2018-08-01T04:40:00.000+0000,19.25,799e39eb2c1b411185424b0f2cd18bda,CUS-3026014945,115.85 -31.9,debit,182446574
9,posted,NaN,ACC-602667573,151.23 -33.96,INTER BANK,NaN,Michael,466.58,2018-08-01,M,40,NaN,NaN,2018-08-01T06:00:00.000+0000,21.00,798a77869014441b840a7a8a23408332,CUS-2031327464,NaN,debit,602667573
